In [1]:
import random
from format_data import get_data_SFTTrainer
from rag import get_RAG_context
# from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from sklearn.feature_extraction.text import TfidfVectorizer


**Data Handling**

In [2]:
train_dataset = get_data_SFTTrainer('data/math/merged_math_problems_train_clean.json')
random.shuffle(train_dataset)

split = 0.8
train_dataset = train_dataset[:int(split*len(train_dataset))]
val_dataset = train_dataset[int(split*len(train_dataset)):]
test_dataset = get_data_SFTTrainer('data/aime_clean.json')

In [4]:
train_dataset
val_dataset
test_dataset
print(len(test_dataset))

975


In [4]:
text_format = """### prompt: {problem_statement} [completion]: {solution}"""

# RAG support
prompts = [example["prompt"] for example in train_dataset] 
vectorizer = TfidfVectorizer()                       # Initialize TF-IDF vectorizer
tfidf_matrix = vectorizer.fit_transform(prompts)     # Fit vectorizer on prompts
RAG_params = {
    "train_data": train_dataset,
    "vectorizer": vectorizer,
    "tfidf_matrix": tfidf_matrix,
}
TOP_K = 3

def formatting_prompts_func(dataset):
    output_texts = []
    for question_dict in dataset:
        problem, solution = question_dict["prompt"], question_dict["completion"]
        text = text_format.format(problem_statement=problem, solution=solution)

        # get RAG-supported context
        context = get_RAG_context(question_dict, RAG_params, top_k=TOP_K)
        output_texts.append(context+text)
    return output_texts


In [5]:
formatting_prompts_func(test_dataset[:1])

["### prompt: We roll a fair 6-sided die 5 times.  What is the probability that we get an odd number in exactly 4 of the 5 rolls? [completion]: The chances of getting an odd or even number are equal, so there are $2^5=32$ equally likely outcomes.  If we want to get exactly 4 of 5 the rolls to be odd, the probability is $ LaTex dfrac{ LaTex binom{5}{4}}{2^5}= LaTex boxed{ LaTex dfrac{5}{32}}.$### prompt: Suppose we roll a standard fair 6-sided die.  What is the probability that a perfect square is rolled? [completion]: A 1 or 4 can be rolled for success, which is 2 out of 6 possible outcomes, so its probability is $ LaTex dfrac26 =  LaTex boxed{ LaTex dfrac13}$.### prompt: If I roll a fair, regular six-sided die four times, what is the probability that I will roll the number $1$ exactly three times? [completion]: Each roll is independent of every other roll, so the probability of getting a $1$ on any given roll is $ LaTex frac{1}{6}$, and the probability of not getting a $1$ on any give

In [6]:
# response_template = " [completion]: "
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

# trainer = SFTTrainer(
#     model,
#     train_dataset=train_dataset,
#     formatting_func=formatting_prompts_func,
#     data_collator=collator,
# )

In [7]:
# trainer.train()